# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [ ]:
# Your code here; import the necessary packages

In [ ]:
# Your code here; create the connection

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [1]:
# Your code here; import the necessary packages
import pandas as pd
from sqlite3 import connect
conn = connect("data.sqlite")

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [4]:
# Your code here
pd.read_sql(
    """
    select
        customerNumber,
        contactLastName,
        contactFirstName
    from customers
    where customerNumber in (
        select customerNumber 
        from orders 
        where 
            orderDate = '2003-01-31'
        );
    """, conn
)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [5]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
pd.read_sql(
    """
    select
        productName, count(distinct customerNumber) as distinctCustomersCount
    from products
    join orderdetails
        using (productCode)
    join orders
        using (orderNumber)
    group by 1
    order by 2 desc
    limit 5;
    """, conn
)


,productName,distinctCustomersCount
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [6]:
# Your code here
pd.read_sql(
    """
    with grouped_products as (
        select
            productCode, count(distinct customerNumber) as distinctCustomersCount
        from products
        join orderdetails
            using (productCode)
        join orders
            using (orderNumber)
        group by 1
        having distinctCustomersCount < 20
    ),

    filtered_employees as (
        select salesRepEmployeeNumber
        from customers
        join orders
            using (customerNumber)
        join orderdetails
            using (orderNumber)
        where productCode in (select productCode from grouped_products)
    )
    
    select
        employeeNumber, firstName, lastName,
        city, officeCode
    from employees
    left join offices
        using (officeCode)
    where 
        employeeNumber in (select * from filtered_employees);
    """, conn
)

,employeeNumber,firstName,lastName,city,officeCode
0,1165,Leslie,Jennings,San Francisco,1
1,1166,Leslie,Thompson,San Francisco,1
2,1188,Julie,Firrelli,Boston,2
3,1216,Steve,Patterson,Boston,2
4,1286,Foon Yue,Tseng,NYC,3
5,1323,George,Vanauf,NYC,3
6,1337,Loui,Bondur,Paris,4
7,1370,Gerard,Hernandez,Paris,4
8,1401,Pamela,Castillo,Paris,4
9,1501,Larry,Bott,London,7


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [7]:
# Your code here
pd.read_sql(
    """
    select
        firstName, lastName, count(*) as numberOfCustomers
    from employees
    join customers
        on employees.employeeNumber = customers.salesRepEmployeeNumber
    where employeeNumber in (
        select salesRepEmployeeNumber
        from (
            select salesRepEmployeeNumber, avg(creditLimit) as avgCreditLimit
            from customers
            group by 1
            having avgCreditLimit > 15000
            )
        )
    group by 1, 2
    
    """, conn
)

,firstName,lastName,numberOfCustomers
0,Andy,Fixter,5
1,Barry,Jones,9
2,Foon Yue,Tseng,7
3,George,Vanauf,8
4,Gerard,Hernandez,7
5,Julie,Firrelli,6
6,Larry,Bott,8
7,Leslie,Jennings,6
8,Leslie,Thompson,6
9,Loui,Bondur,6


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!